# Q1. We saw the perceptron algorithm in Lecture 15. I want you to implement this algorithm, and use it to classify banknotes as forged or authentic in this dataset. In particular,

# (a) Code up the perceptron algorithm described on slide 7 of Lecture 15 using the same notation as in the slides. [10 points]

## Here there are two ways in which we are trying to check the convergence:
1. Maximum iterations are reached
2. If the weights are not updated contunously for k times


In [1]:
def train(X, y, max_iter=1000, tol = 1e-04, k=10):
    w=np.ones((X.shape[1],1)) # initialiazing the weight vector to all 1s
    #b=1
    learning_rate=1
    ctr=0
    X = np.array(X)
    y = np.array(y)
    for it in range(max_iter):
        i = np.random.randint(X.shape[0]) 
        XX=X[i,]#randomly choosing a data point 
        yy=y[i]#randomly choosing a data point 
    #    print(XX, yy, it)
        if yy * (np.dot(w.T,XX))<0:
            ctr=0
            tmp = w
            w+=learning_rate * np.dot(XX,yy).reshape(4,1)
            #print(w-tmp, it)
            if np.all(w!=tmp) and np.all(np.abs(w-tmp)<= tol) :
                #print("aaya")
                break
        else:
            ctr+=1
        if ctr==k:
            break
    return w,it

# (b) Write functions to make predictions using the algorithm for the banknotes dataset. Preprocess the dataset to handle missing and anomalous data. [10 points]

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"data_banknote_authentication.txt")
df.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'class']
df.reset_index(drop=True, inplace=True)

#performing min-max scaling on the data
df['variance'] = (df['variance']- df['variance'].min())/(df['variance'].max()- df['variance'].min())
df['skewness'] = (df['skewness']- df['skewness'].min())/(df['skewness'].max()- df['skewness'].min())
df['curtosis'] = (df['curtosis']- df['curtosis'].min())/(df['curtosis'].max()- df['curtosis'].min())
df['entropy'] = (df['entropy']- df['entropy'].min())/(df['entropy'].max()- df['entropy'].min())
#df
#print(len(df))
df.drop_duplicates(['variance', 'skewness', 'curtosis', 'entropy'], inplace=True) #dropping unnecessary duplicates
#print(len(df))
df['class'] = df['class'].apply(lambda x: -1 if x == 0 else 1)

df.head(5)

,variance,skewness,curtosis,entropy,class
0,0.835659,0.820982,0.121804,0.644326,-1
1,0.786629,0.416648,0.310608,0.786951,-1
2,0.757105,0.871699,0.054921,0.450440,-1
3,0.531578,0.348662,0.424662,0.687362,-1
4,0.822859,0.877275,0.057100,0.489711,-1


## A function to classify if the notes are authentic or not:

In [3]:
def predict(X,w):
    y_pred = np.zeros(X.shape[0])

    for i in range(X.shape[0]):
        y_pred[i] = np.where(np.dot(w.T,X[i])<0, -1, 1)
    return y_pred

# (c) Train the algorithm on the dataset using cross-validation and report cross-validated test set error [10 points]

## Performing 10 fold cross validation, and choosing the set of weights which give the minimum validation set (chosen randomly in every iteration) error, 

In [4]:
def cross_validate():
    min_err = 1
    best_w = np.ones(4)#initializing the weight vectors as all 1s
    s = 0
    for i in range(10):
        #print(i)
        training_data_u = df.sample(frac=0.8)
        #testing_data_u = df.drop(training_data_u.index)

        validation_data_u = training_data_u.sample(frac=0.25)
        training_data_u = training_data_u.drop(validation_data_u.index)

        X_train = training_data_u[['variance', 'skewness', 'curtosis', 'entropy']]
        y_train = training_data_u['class']

        w,it = train(X_train, y_train)

        X_validate = validation_data_u[['variance', 'skewness', 'curtosis', 'entropy']]
        y_validate = validation_data_u['class']

        X_validate = np.array(X_validate)
        y_validate = np.array(y_validate)
        
        #print(X_validate,y_validate )
        y_pred = predict(X_validate,w)
        count = 0
       # print(y_pred)
        for i in range(len(y_pred)):
            if(y_pred[i]==y_validate[i]):
                count+=1

        error = 1-count/len(y_pred)
        s+=error
        #print("Error:", error )
        if(error<min_err):
            min_err = error
            best_w = w
        
        
        print("\nCurrent Error:", round(error*100,3), '%',"\tCurrent Weigths:", w.tolist())
        #break
    print("\n\n\nMinimum reported test set error after 10 fold cross validation:", round(min_err*100,3), '%')

    print("\nAverage test set error:", round(s*10,3), '%')
    return best_w

best_weights = cross_validate()
        


Current Error: 26.296 % 	Current Weigths: [[-3.1919436211409917], [0.5622002192728074], [-0.3945451138346219], [2.6879593005810296]]

Current Error: 27.407 % 	Current Weigths: [[-2.8592237630616797], [0.1513730743469519], [0.4608788420531158], [1.0539649199378052]]

Current Error: 17.037 % 	Current Weigths: [[-2.542707526556044], [-0.39211964961253076], [-0.35951093113920846], [1.912295661820199]]

Current Error: 30.741 % 	Current Weigths: [[-1.526788972300947], [-1.6256504282555095], [0.22419152648243487], [1.317475290287969]]

Current Error: 20.741 % 	Current Weigths: [[-1.8321722951777253], [-0.10437804727461852], [0.9362911667779525], [1.2057674786546275]]

Current Error: 25.556 % 	Current Weigths: [[-3.757500587730492], [1.210424411978431], [-0.055162039330562435], [2.1756980095838214]]

Current Error: 20.37 % 	Current Weigths: [[-1.713452105373227], [-0.5129838688554037], [0.38047386219225876], [1.4417440919464983]]

Current Error: 28.519 % 	Current Weigths: [[-1.55227769003887]

# Ensure you use a held out validation set and report F1 score on the held out set for your best model [5 points]



$$Precision = \frac{tp}{tp+fp}$$
$$Recall = \frac{tp}{tp+fn}$$
$$F_{1} \;Score=\frac{2 \times \text{Precisión} \times \text{Recall}}{\text{Precisión} + \text{Recall}}$$
<br>
where,

tp = #true_positives
<br>
fp = #false_positives
<br>
fn = #false_negatives

In [5]:

def compute_params(y_og, y_predicted): #computing true positives, true negatives, false positives, false negatives
	tp = sum((y_og == 1) & (y_predicted == 1))
	tn = sum((y_og == -1) & (y_predicted == -1))
	fp = sum((y_og == -1) & (y_predicted == 1))
	fn = sum((y_og == 1) & (y_predicted == -1))
	return tp, tn, fp, fn

def compute_prec(tp, fp): #function to calculate the precision
	return (tp)/ float( tp + fp)
    
def compute_rec(tp, fn): #function to calculate the recall
	return (tp)/ float( tp + fn)

def compute_f1(y_og, y_pred): #function to calculate the F1 score
    tp, tn, fp, fn = compute_params(y_og, y_pred)
    prec = compute_prec(tp, fp)
    rec = compute_rec(tp, fn)
    f1_score = (2*prec*rec)/ (prec + rec)
    return f1_score


## Dividing overall data into train, test and validation sets. The test set so calculated will be used as the "held-out" validation set later.

In [6]:
training_data_master = df.sample(frac=0.8)
testing_data_master = df.drop(training_data_master.index)

validation_data_master = training_data_master.sample(frac=0.25)
training_data_master = training_data_master.drop(validation_data_master.index)

## Below are the best weights obtained after cross validation:

In [7]:
best_weights

array([[-2.96465577],
       [ 0.66199442],
       [-0.0064637 ],
       [ 1.60624824]])

In [8]:
X_test = testing_data_master[['variance', 'skewness', 'curtosis', 'entropy']]
y_test = testing_data_master['class']

X_test = np.array(X_test)
y_test = np.array(y_test)

y_pred = predict(X_test,best_weights)

print('The required F1 score for the held out validation set is', round(compute_f1(y_test, y_pred),4))

The required F1 score for the held out validation set is 0.8724
